In [1]:
import backtrader as bt
import pandas as pd
from numpy import *
%matplotlib inline
import backtrader.analyzers as btanalyzers

df = pd.read_csv('BTCUSDT3.csv',index_col=0)
df.columns = ['open_time','open','high','low','close','volume','close_time']
df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
df = df.set_index('open_time')
df = df.resample('15min').last()

In [2]:
feeds = bt.feeds.PandasDirectData(
    name='BTC', # 多股回测时用户区分数据对象
    timeframe = bt.TimeFrame.Minutes,
    dataname=df,
#     datetime=0,  # 日期行所在列
    dtformat=('%Y-%m-%d %H:%M:%S'), # 日期和时间格式
    tmformat=('%H:%M:%S'), # 时间格式
    open=1,  # 开盘价所在列
    high=2,  # 最高价所在列
    low=3,  # 最低价所在列
    close=4,  # 收盘价价所在列
    volume=5,  # 成交量所在列
    openinterest=-1,  # 无未平仓量列.(openinterest是期货交易使用的)
)

In [3]:
# 创建一个策略
pnl = pd.DataFrame()
value_lis = []
date_lis = []

sharpe_lis = []
class DualThrust(bt.Strategy):
    # 需要调优的参数
    params = (
        ('N', 7),
        ('K',0.5),
        ('stop_loss',2),
        ('stop_gain',100)
        )
    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.close_p = self.datas[0].close
        self.high = self.datas[0].high
        self.low  = self.datas[0].low
        self.open = self.datas[0].open
        self.N = self.params.N
        self.K1 = self.params.K
        self.K2 = self.params.K
        self.stop_loss = self.params.stop_loss/100
        self.stop_gain = self.params.stop_gain/100
        
        self.high_lis = []
        self.low_lis = []
        self.close_p_lis = []
        self.single_day_high_lis = []
        self.single_day_low_lis = []
        
        self.day_counter = 0
        self.start_strategy = False
        
        self.BuyLine = None
        self.SellLine = None
        self.value_perday = []
        
    
    def prenext(self):
        pass
    
    def next(self):
#         self.log(self.close_p.get(ago=-1,size=20))
#         self.stop_price = mean(self.close_p.get(ago=-1,size=120))
#         self.avg_close = mean(self.close_p.get(ago=-1,size=1440*7))
        date = self.datas[0].datetime.datetime(0)
        day = str(date).split(' ')[0]
        time = str(date).split(' ')[1]
        size = self.getposition(data = self.datas[0]).size
        
        self.single_day_high_lis.append(self.high[0])
        self.single_day_low_lis.append(self.low[0])
#         self.std_sample.append(self.close_p[0])
        
        if time == '00:00:00':
            self.day_counter += 1
            self.open_ = self.open[0]
            self.close_p_lis.append(self.close_p[0])
            self.high_lis.append(max(self.single_day_high_lis))
            self.low_lis.append(min(self.single_day_low_lis))
            self.value_perday.append(self.broker.getvalue())
            
            self.single_day_high_lis = []
            self.single_day_low_lis = []
            
            if self.day_counter == self.N:
                # 计算各个指标
                HH = max(self.high_lis)
                LC = min(self.close_p_lis)
                HC = max(self.close_p_lis)
                LL = min(self.low_lis)
                Range = max(HH-LC,HC-LL)
                

                self.BuyLine = self.open_ + self.K1 * Range
                self.SellLine = self.open_ - self.K2 * Range
                # 去除第一个保持长度为N
                self.high_lis.remove(self.high_lis[0])
                self.low_lis.remove(self.low_lis[0])
                self.close_p_lis.remove(self.close_p_lis[0])
                self.start_strategy = True
                self.day_counter -= 1
                self.close()
                self.log('收盘平仓')
                self.no_buy = False
                self.no_sell = False
#                 std_dic[std(std_sample)] = []
        if self.start_strategy:
            cross_upper = self.high[0] - self.BuyLine > 0
            cross_lower = self.low[0] - self.SellLine < 0
            
            if size != 0:
                unrealized_pnl = self.close_p[0]/self.position.price - 1    
                if (size > 0 and unrealized_pnl <= -self.stop_loss) or (size < 0 and unrealized_pnl >= self.stop_loss):
                    self.close()
                    self.log('止损平仓')
                    unrealized_pnl=0
                
                    
            if cross_upper and size <= 0:
                self.close()
                self.log(self.getposition(data = self.datas[0]).size)
                self.order_target_percent(target=0.9)
                self.log('-'*50+'买入开多'+'-'*50)
                self.log(self.getposition(data = self.datas[0]).size)

            if cross_lower and size >= 0:
                self.close()
                self.log(self.getposition(data = self.datas[0]).size)
                self.order_target_percent(target=-0.9)
                self.log('-'*50+'卖出做空'+'-'*50)
                self.log(self.getposition(data = self.datas[0]).size)
                
        
        # 获取当天value
        value = self.broker.getvalue()
        date_lis.append(date)
        value_lis.append(value)
        
    def stop(self):
        return_perday = [self.value_perday[x]/self.value_perday[x-1] - 1 for x in range(len(self.value_perday)) if x != 0]
        mean_v = mean(return_perday)
        std_v = std(return_perday)
        sharpe = (mean_v - 0.04/365)/std_v * 365 ** 0.5
        sharpe_lis.append(sharpe)

# 参数调优

In [4]:
class CommInfoFractional(bt.CommissionInfo):
    def getsize(self, price, cash):
        '''Returns fractional size for cash operation @price'''
        return self.p.leverage * (cash / price)

cerebro = bt.Cerebro()
cerebro.broker.setcash(10000.0)
print('Starting Portfolio Value: %.5f' % cerebro.broker.getvalue())
cerebro.adddata(feeds)
cerebro.broker.setcommission(commission=0.0004)

cerebro.optstrategy(
    strategy=DualThrust,
    N=[3, 7],
    K = [0.5],
    stop_loss = [0.2]
)

commission = 0.001
comminfo = CommInfoFractional(commission=commission)
cerebro.broker.addcommissioninfo(comminfo)
cerebro.addanalyzer(btanalyzers.DrawDown, _name = "drawdown")
cerebro.addanalyzer(btanalyzers.Returns, _name = "returns",timeframe=bt.TimeFrame.Minutes)
result = cerebro.run(maxcpus=1)
print('Final Portfolio Value: %.5f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00000
2021-11-12, 收盘平仓
2021-11-12, 0
2021-11-12, --------------------------------------------------卖出做空--------------------------------------------------
2021-11-12, 0
2021-11-12, 止损平仓
2021-11-13, 收盘平仓
2021-11-14, 收盘平仓
2021-11-15, 收盘平仓
2021-11-15, 0.0
2021-11-15, --------------------------------------------------卖出做空--------------------------------------------------
2021-11-15, 0.0
2021-11-15, 止损平仓
2021-11-15, 0.0
2021-11-15, --------------------------------------------------卖出做空--------------------------------------------------
2021-11-15, 0.0
2021-11-15, 止损平仓
2021-11-15, 0.0
2021-11-15, --------------------------------------------------卖出做空--------------------------------------------------
2021-11-15, 0.0
2021-11-16, 收盘平仓
2021-11-16, 0.0
2021-11-16, --------------------------------------------------卖出做空--------------------------------------------------
2021-11-16, 0.0
2021-11-16, 止损平仓
2021-11-16, 0.0
2021-11-16, ----------------------------------------

2022-01-17, 收盘平仓
2022-01-17, 0.0
2022-01-17, --------------------------------------------------卖出做空--------------------------------------------------
2022-01-17, 0.0
2022-01-17, 止损平仓
2022-01-17, 0.0
2022-01-17, --------------------------------------------------卖出做空--------------------------------------------------
2022-01-17, 0.0
2022-01-18, 收盘平仓
2022-01-18, 0.0
2022-01-18, --------------------------------------------------卖出做空--------------------------------------------------
2022-01-18, 0.0
2022-01-18, 止损平仓
2022-01-18, 0.0
2022-01-18, --------------------------------------------------卖出做空--------------------------------------------------
2022-01-18, 0.0
2022-01-18, 止损平仓
2022-01-18, 0.0
2022-01-18, --------------------------------------------------卖出做空--------------------------------------------------
2022-01-18, 0.0
2022-01-18, 止损平仓
2022-01-18, 0.0
2022-01-18, --------------------------------------------------卖出做空--------------------------------------------------
2022-01-18, 0.0
2022

2022-03-26, 收盘平仓
2022-03-27, 收盘平仓
2022-03-27, 0.0
2022-03-27, --------------------------------------------------买入开多--------------------------------------------------
2022-03-27, 0.0
2022-03-28, 收盘平仓
2022-03-29, 收盘平仓
2022-03-30, 收盘平仓
2022-03-31, 收盘平仓
2022-03-31, 0.0
2022-03-31, --------------------------------------------------卖出做空--------------------------------------------------
2022-03-31, 0.0
2022-03-31, 止损平仓
2022-03-31, 0.0
2022-03-31, --------------------------------------------------卖出做空--------------------------------------------------
2022-03-31, 0.0
2022-04-01, 收盘平仓
2022-04-01, 0.0
2022-04-01, --------------------------------------------------卖出做空--------------------------------------------------
2022-04-01, 0.0
2022-04-01, 止损平仓
2022-04-02, 收盘平仓
2022-04-03, 收盘平仓
2022-04-03, 0.0
2022-04-03, --------------------------------------------------买入开多--------------------------------------------------
2022-04-03, 0.0
2022-04-03, 止损平仓
2022-04-04, 收盘平仓
2022-04-04, 0.0
2022-04-04, ------

2022-05-30, 收盘平仓
2022-05-30, 0.0
2022-05-30, --------------------------------------------------买入开多--------------------------------------------------
2022-05-30, 0.0
2022-05-31, 收盘平仓
2022-06-01, 收盘平仓
2022-06-01, 0.0
2022-06-01, --------------------------------------------------卖出做空--------------------------------------------------
2022-06-01, 0.0
2022-06-01, 止损平仓
2022-06-01, 0.0
2022-06-01, --------------------------------------------------卖出做空--------------------------------------------------
2022-06-01, 0.0
2022-06-01, 止损平仓
2022-06-01, 0.0
2022-06-01, --------------------------------------------------卖出做空--------------------------------------------------
2022-06-01, 0.0
2022-06-02, 收盘平仓
2022-06-03, 收盘平仓
2022-06-04, 收盘平仓
2022-06-05, 收盘平仓
2022-06-06, 收盘平仓
2022-06-06, 0.0
2022-06-06, --------------------------------------------------买入开多--------------------------------------------------
2022-06-06, 0.0
2022-06-07, 收盘平仓
2022-06-07, 0.0
2022-06-07, ----------------------------------------

2022-08-09, 0.0
2022-08-09, --------------------------------------------------卖出做空--------------------------------------------------
2022-08-09, 0.0
2022-08-09, 止损平仓
2022-08-09, 0.0
2022-08-09, --------------------------------------------------卖出做空--------------------------------------------------
2022-08-09, 0.0
2022-08-09, 止损平仓
2022-08-10, 收盘平仓
2022-08-10, 0.0
2022-08-10, --------------------------------------------------买入开多--------------------------------------------------
2022-08-10, 0.0
2022-08-10, 止损平仓
2022-08-10, 0.0
2022-08-10, --------------------------------------------------买入开多--------------------------------------------------
2022-08-10, 0.0
2022-08-10, 止损平仓
2022-08-10, 0.0
2022-08-10, --------------------------------------------------买入开多--------------------------------------------------
2022-08-10, 0.0
2022-08-10, 止损平仓
2022-08-10, 0.0
2022-08-10, --------------------------------------------------买入开多--------------------------------------------------
2022-08-10, 0.0
2022

2022-09-30, 收盘平仓
2022-09-30, 0.0
2022-09-30, --------------------------------------------------买入开多--------------------------------------------------
2022-09-30, 0.0
2022-09-30, 止损平仓
2022-10-01, 收盘平仓
2022-10-02, 收盘平仓
2022-10-03, 收盘平仓
2022-10-03, 0.0
2022-10-03, --------------------------------------------------买入开多--------------------------------------------------
2022-10-03, 0.0
2022-10-03, 止损平仓
2022-10-03, 0.0
2022-10-03, --------------------------------------------------买入开多--------------------------------------------------
2022-10-03, 0.0
2022-10-04, 收盘平仓
2022-10-04, 0.0
2022-10-04, --------------------------------------------------买入开多--------------------------------------------------
2022-10-04, 0.0
2022-10-04, 止损平仓
2022-10-04, 0.0
2022-10-04, --------------------------------------------------买入开多--------------------------------------------------
2022-10-04, 0.0
2022-10-04, 止损平仓
2022-10-04, 0.0
2022-10-04, --------------------------------------------------买入开多--------------------

2021-12-13, 收盘平仓
2021-12-13, 0.0
2021-12-13, --------------------------------------------------卖出做空--------------------------------------------------
2021-12-13, 0.0
2021-12-13, 止损平仓
2021-12-13, 0.0
2021-12-13, --------------------------------------------------卖出做空--------------------------------------------------
2021-12-13, 0.0
2021-12-13, 止损平仓
2021-12-13, 0.0
2021-12-13, --------------------------------------------------卖出做空--------------------------------------------------
2021-12-13, 0.0
2021-12-14, 收盘平仓
2021-12-15, 收盘平仓
2021-12-16, 收盘平仓
2021-12-17, 收盘平仓
2021-12-17, 0.0
2021-12-17, --------------------------------------------------卖出做空--------------------------------------------------
2021-12-17, 0.0
2021-12-17, 止损平仓
2021-12-18, 收盘平仓
2021-12-19, 收盘平仓
2021-12-20, 收盘平仓
2021-12-21, 收盘平仓
2021-12-21, 0.0
2021-12-21, --------------------------------------------------买入开多--------------------------------------------------
2021-12-21, 0.0
2021-12-21, 止损平仓
2021-12-21, 0.0
2021-12-21, ------

2022-04-08, 收盘平仓
2022-04-09, 收盘平仓
2022-04-10, 收盘平仓
2022-04-11, 收盘平仓
2022-04-11, 0.0
2022-04-11, --------------------------------------------------卖出做空--------------------------------------------------
2022-04-11, 0.0
2022-04-11, 止损平仓
2022-04-12, 收盘平仓
2022-04-13, 收盘平仓
2022-04-14, 收盘平仓
2022-04-15, 收盘平仓
2022-04-16, 收盘平仓
2022-04-17, 收盘平仓
2022-04-18, 收盘平仓
2022-04-19, 收盘平仓
2022-04-20, 收盘平仓
2022-04-21, 收盘平仓
2022-04-22, 收盘平仓
2022-04-23, 收盘平仓
2022-04-24, 收盘平仓
2022-04-25, 收盘平仓
2022-04-26, 收盘平仓
2022-04-26, 0.0
2022-04-26, --------------------------------------------------卖出做空--------------------------------------------------
2022-04-26, 0.0
2022-04-27, 收盘平仓
2022-04-28, 收盘平仓
2022-04-29, 收盘平仓
2022-04-29, 0.0
2022-04-29, --------------------------------------------------卖出做空--------------------------------------------------
2022-04-29, 0.0
2022-04-29, 止损平仓
2022-04-30, 收盘平仓
2022-05-01, 收盘平仓
2022-05-02, 收盘平仓
2022-05-03, 收盘平仓
2022-05-04, 收盘平仓
2022-05-04, 0.0
2022-05-04, --------------------------------

2022-07-30, 收盘平仓
2022-07-31, 收盘平仓
2022-08-01, 收盘平仓
2022-08-02, 收盘平仓
2022-08-03, 收盘平仓
2022-08-04, 收盘平仓
2022-08-05, 收盘平仓
2022-08-06, 收盘平仓
2022-08-07, 收盘平仓
2022-08-08, 收盘平仓
2022-08-08, 0.0
2022-08-08, --------------------------------------------------买入开多--------------------------------------------------
2022-08-08, 0.0
2022-08-09, 收盘平仓
2022-08-09, 0.0
2022-08-09, --------------------------------------------------卖出做空--------------------------------------------------
2022-08-09, 0.0
2022-08-09, 止损平仓
2022-08-10, 收盘平仓
2022-08-10, 0.0
2022-08-10, --------------------------------------------------买入开多--------------------------------------------------
2022-08-10, 0.0
2022-08-10, 止损平仓
2022-08-10, 0.0
2022-08-10, --------------------------------------------------买入开多--------------------------------------------------
2022-08-10, 0.0
2022-08-10, 止损平仓
2022-08-10, 0.0
2022-08-10, --------------------------------------------------买入开多--------------------------------------------------
2022-08-10, 0.0


2022-11-12, 收盘平仓
2022-11-13, 收盘平仓
Final Portfolio Value: 9728.72855


In [6]:
par_list = [[x[0].params.N,
             x[0].params.K,
             x[0].params.stop_loss,
            x[0].analyzers.returns.get_analysis()['rtot'], 
            x[0].analyzers.drawdown.get_analysis()['max']['drawdown']/100,
                ] for x in result]

par_df = pd.DataFrame(par_list, columns = ['N', 'K', 'stop_loss', 'return', 'max_drawdown'])
par_df['sharpe'] = sharpe_lis
par_df

,N,K,stop_loss,return,max_drawdown,sharpe
0,3,0.5,0.2,-0.045940,0.319681,-0.172462
1,7,0.5,0.2,-0.027502,0.162234,-0.315973
